# Sharing Ephemerides between GMAT and MONTE
GMAT and MONTE both provide users with the ability to write the position and velocity at a sequence of times to a file. These ephemerides can take several forms, ranging from Monte's Binary Object Archive (boa) to GMAT's support for STK ephemerides and CCSDS Orbit Ephemeris Messages (OEMs). Both programs share the SPICE SPK format in common, so that is the ephemeris format used to share trajectory data between the systems.

  

In [ ]:
import Monte as M
import mpy.io.data as defaultData
import mpy.traj.force.grav.basic as basicGrav
from mpy.units import *
import cristo

In [ ]:
from load_gmat import *

In [ ]:
# Load planetary ephemeris de405 from default data
boa = defaultData.load( "ephem/planet/de405" )

In [ ]:
# Build the gravity nodes connecting the spacecraft to the gravitational
# bodies we want active.
scName = "mySc"
basicGrav.add( boa, scName, [ "Sun", "Earth", "Moon" ] )

In [ ]:
# Define an initial state of a spacecraft using Conic elements
radius = 6778.0 * km
state = M.State(
   boa, scName, 'Earth',
   M.Conic.semiMajorAxis( radius ),
   M.Conic.eccentricity( 0.0 ),
   M.Conic.inclination( 5 * deg ),
   M.Conic.argumentOfLatitude( 0 * deg),
   M.Conic.longitudeOfNode( 0 * deg),
   M.Conic.trueAnomaly( 0 * deg)
   )

In [ ]:
# Define which forces will act on the spacecraft during propagation.
forces = [
   M.GravityForce( boa, scName ),
   ]

In [ ]:
# Set up the beginning and end times for our scenario.
beginTime = M.Epoch( "01-JAN-2000 00:00:00 ET" )
endTime = M.Epoch( "10-JAN-2000 00:00:00 ET" )

In [ ]:
# Add the initial state to the "IntegState"
integState = M.IntegState(
   boa,         # Model database used in integration
   beginTime,   # Start time
   endTime,     # End time
   [],          # Events to trigger integration end (none)
   scName,      # Spacecraft name
   'Earth',     # Center body
   'EME2000',   # Input frame
   'EME2000',   # Integration frame
   state,       # State initial conditions
   forces,      # Forces which act on state
   False,       # Integrate only partial derivatives (false)
   [],          # Parameters to be used in partial derivative calculations (none)
   []           # Partials tolerance scale factors (allows different partial
                # derivatives to have different integration tolerances, none)
   )

# Add state to our propagation manager "IntegSetup"
integ = M.IntegSetup( boa )
integ.add( integState )

In [ ]:
# Set up the propagator.
prop = M.DivaPropagator( boa, "DIVA", integ )

prop.create( boa, beginTime, endTime )

In [ ]:
# Write SPK
M.SpiceName.bodyInsert( -42, "mySc" )

cristo.convert( boa, "MySc.bsp" )